In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle as w
import explore as e
import model as m

from scipy.stats import ttest_1samp
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

%matplotlib inline

In [2]:
df = w.wrangle_new_york()
train, val, test = w.split_new_york(df)
X_train, y_train, X_val, y_val, X_test, y_test = m.X_y_modeling_split(train, val, test, 'avg_temp')


In [3]:
def wrangle_new_york(): 

    df = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv')
    df = df[df['City'] == 'New York']
    df = df.dropna()
    df.dt = pd.to_datetime(df.dt)
    df = df.rename(columns = {
                             'dt' : 'date',
                             'AverageTemperature': 'avg_temp',
                             'AverageTemperatureUncertainty': 'avg_temp_uncertainty'
                             })
    df = df.drop(['Latitude', 'Longitude', 'Country', 'City'], axis = 1)
    df = df.set_index('date')
    df.index.sort_values()
    df = df[df.index > '1923-01-01']
    #df = df[df.index > '2000-01-01']
    df = df.drop(columns= ['avg_temp_uncertainty'])

    return df

In [4]:
df = wrangle_new_york()

In [5]:
df

,avg_temp
date,
1923-02-01,-5.401
1923-03-01,1.233
1923-04-01,8.841
1923-05-01,14.028
1923-06-01,21.221
...,...
2013-05-01,15.544
2013-06-01,20.892
2013-07-01,24.722


In [6]:
def rolling_avg_model(train, val):
    period = 365

    avg_temp = round(train['avg_temp'].rolling(period).mean().iloc[-1], 4)

    # yhat_df = make_predictions()

    yhat_df = pd.DataFrame({'avg_temp': [avg_temp]}, index = val.index)
    yhat_df.head()

    return yhat_df


In [7]:
yhat_df = rolling_avg_model(train, val)

In [8]:
#yhat_df = m.rolling_avg_model(train, val)

In [9]:
print(yhat_df)

            avg_temp
date                
2007-09-01   10.4364
2007-10-01   10.4364
2007-11-01   10.4364
2007-12-01   10.4364
2008-01-01   10.4364
2008-02-01   10.4364
2008-03-01   10.4364
2008-04-01   10.4364
2008-05-01   10.4364
2008-06-01   10.4364
2008-07-01   10.4364
2008-08-01   10.4364
2008-09-01   10.4364
2008-10-01   10.4364
2008-11-01   10.4364
2008-12-01   10.4364
2009-01-01   10.4364
2009-02-01   10.4364
2009-03-01   10.4364
2009-04-01   10.4364
2009-05-01   10.4364
2009-06-01   10.4364
2009-07-01   10.4364
2009-08-01   10.4364
2009-09-01   10.4364
2009-10-01   10.4364
2009-11-01   10.4364
2009-12-01   10.4364
2010-01-01   10.4364
2010-02-01   10.4364
2010-03-01   10.4364
2010-04-01   10.4364
2010-05-01   10.4364
2010-06-01   10.4364
2010-07-01   10.4364
2010-08-01   10.4364
2010-09-01   10.4364
2010-10-01   10.4364
2010-11-01   10.4364
2010-12-01   10.4364
2011-01-01   10.4364
2011-02-01   10.4364
2011-03-01   10.4364
2011-04-01   10.4364
2011-05-01   10.4364
2011-06-01   

In [10]:
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(val[target_var], yhat_df[target_var])), 0)
    return rmse

In [11]:
evaluate('avg_temp')

9.0